In [ ]:
pip install Flask


In [ ]:
from flask import Flask, render_template, request, jsonify
import pandas as pd
import numpy as np
import pickle

app = Flask(__name__)

# Load the trained model (make sure you have a trained model saved as 'model.pkl')
model = pickle.load(open('model.pkl', 'rb'))
scaler = pickle.load(open('scaler.pkl', 'rb'))

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    if request.method == 'POST':
        data = request.form.to_dict()

        # Convert data to DataFrame
        df = pd.DataFrame(data, index=[0])

        # Data preprocessing steps (ensure the same preprocessing as in training)
        df['Item_Weight'] = df['Item_Weight'].astype(float)
        df['Outlet_Years'] = 2023 - int(df['Outlet_Establishment_Year'])
        df.drop(['Outlet_Establishment_Year'], axis=1, inplace=True)

        # Apply label encoding (must match the encoding used during training)
        label_encoders = pickle.load(open('label_encoders.pkl', 'rb'))
        categorical_features = ['Item_Fat_Content', 'Item_Type', 'Outlet_Identifier', 'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type']
        for feature in categorical_features:
            df[feature] = label_encoders[feature].transform(df[feature])

        # Apply scaling
        df_scaled = scaler.transform(df)

        # Predict
        prediction = model.predict(df_scaled)

        return jsonify({'prediction': float(prediction[0])})

if __name__ == '__main__':
    app.run(debug=True)


In [ ]:
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor

# Assuming you have already trained your model as in the previous example
# and saved it as 'model.pkl' and 'scaler.pkl'

# Save the model
with open('model.pkl', 'wb') as f:
    pickle.dump(best_model, f)

# Save the scaler
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

# Save the label encoders
label_encoders = {
    'Item_Fat_Content': le,
    'Item_Type': le,
    'Outlet_Identifier': le,
    'Outlet_Size': le,
    'Outlet_Location_Type': le,
    'Outlet_Type': le
}
with open('label_encoders.pkl', 'wb') as f:
    pickle.dump(label_encoders, f)


In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>BigMart Sales Prediction</title>
    <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.3.1/css/bootstrap.min.css">
</head>
<body>
    <div class="container">
        <h1 class="mt-5">BigMart Sales Prediction</h1>
        <form id="predictionForm">
            <div class="form-group">
                <label for="Item_Weight">Item Weight</label>
                <input type="text" class="form-control" id="Item_Weight" name="Item_Weight">
            </div>
            <div class="form-group">
                <label for="Item_Fat_Content">Item Fat Content</label>
                <input type="text" class="form-control" id="Item_Fat_Content" name="Item_Fat_Content">
            </div>
            <div class="form-group">
                <label for="Item_Type">Item Type</label>
                <input type="text" class="form-control" id="Item_Type" name="Item_Type">
            </div>
            <div class="form-group">
                <label for="Outlet_Identifier">Outlet Identifier</label>
                <input type="text" class="form-control" id="Outlet_Identifier" name="Outlet_Identifier">
            </div>
            <div class="form-group">
                <label for="Outlet_Size">Outlet Size</label>
                <input type="text" class="form-control" id="Outlet_Size" name="Outlet_Size">
            </div>
            <div class="form-group">
                <label for="Outlet_Location_Type">Outlet Location Type</label>
                <input type="text" class="form-control" id="Outlet_Location_Type" name="Outlet_Location_Type">
            </div>
            <div class="form-group">
                <label for="Outlet_Type">Outlet Type</label>
                <input type="text" class="form-control" id="Outlet_Type" name="Outlet_Type">
            </div>
            <div class="form-group">
                <label for="Outlet_Establishment_Year">Outlet Establishment Year</label>
                <input type="text" class="form-control" id="Outlet_Establishment_Year" name="Outlet_Establishment_Year">
            </div>
            <button type="submit" class="btn btn-primary">Predict</button>
        </form>
        <h3 class="mt-5" id="predictionResult"></h3>
    </div>

    <script src="https://code.jquery.com/jquery-3.3.1.min.js"></script>
    <script>
        $(document).ready(function() {
            $('#predictionForm').on('submit', function(event) {
                event.preventDefault();
                $.ajax({
                    url: '/predict',
                    method: 'POST',
                    data: $(this).serialize(),
                    success: function(response) {
                        $('#predictionResult').html('Predicted Sales: ' + response.prediction.toFixed(2));
                    },
                    error: function(error) {
                        console.log(error);
                    }
                });
            });
        });
    </script>
</body>
</html>


In [ ]:
python app.py
